In [1]:
import re
import sys
import glob
import math
import numpy as np
import pandas as pd
import seaborn as sns
from time import ctime
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
sns.set_style('darkgrid')

In [2]:
train_frags = glob.glob("./train_fft/*")
test_frags = glob.glob("./test_fft/*")
train = pd.read_csv("train.csv")
Y = pd.Series(0, index=np.arange(len(train_frags)))

i = 0
for file in train_frags:
    start = './train_fft/'
    end = '.csv'
    seg_id = file[file.find(start)+len(start):file.rfind(end)]
    t2e = int(train.loc[train['segment_id'] == int(seg_id)]['time_to_eruption'].values)
    Y.iloc[i] = t2e
    i = i + 1

test_id = pd.Series(0, index=np.arange(len(test_frags)))    
i = 0
for file in test_frags:
    start = './test_fft/'
    end = '.csv'
    seg_id = file[file.find(start)+len(start):file.rfind(end)]

    test_id.iloc[i] = seg_id
    i = i + 1


In [3]:
from sklearn import preprocessing
timescaler = preprocessing.StandardScaler().fit(Y.to_numpy().reshape(-1,1))
Y = pd.Series(timescaler.transform(Y.to_numpy().reshape(-1,1)).reshape(-1))

In [4]:
all_pca = pd.read_csv("pca_30.csv")

In [5]:
X = all_pca[:len(train_frags)]
X_test = all_pca[-len(test_frags):]

In [6]:
from sklearn.linear_model._bayes import ARDRegression
from sklearn.ensemble._weight_boosting import AdaBoostRegressor
from sklearn.linear_model._bayes import BayesianRidge
from sklearn.tree._classes import DecisionTreeRegressor 
from sklearn.linear_model._coordinate_descent import ElasticNetCV
from sklearn.ensemble._forest import ExtraTreesRegressor
from sklearn.gaussian_process._gpr import GaussianProcessRegressor
from sklearn.linear_model._glm.glm import GeneralizedLinearRegressor
from sklearn.ensemble._gb import GradientBoostingRegressor
from sklearn.ensemble._hist_gradient_boosting.gradient_boosting import HistGradientBoostingRegressor
from sklearn.linear_model._huber import HuberRegressor
from sklearn.isotonic import IsotonicRegression 
from sklearn.neighbors._regression import KNeighborsRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model._least_angle import LarsCV
from sklearn.linear_model._coordinate_descent import LassoCV
from sklearn.linear_model._least_angle import LassoLarsCV
from sklearn.linear_model._least_angle import LassoLarsIC
from sklearn.linear_model._base import LinearRegression
from sklearn.svm._classes import LinearSVR
from sklearn.neural_network._multilayer_perceptron import MLPRegressor
from sklearn.svm._classes import NuSVR
from sklearn.linear_model._omp import OrthogonalMatchingPursuitCV
from sklearn.linear_model._passive_aggressive import PassiveAggressiveRegressor
from sklearn.neighbors._regression import RadiusNeighborsRegressor
from sklearn.ensemble._forest import RandomForestRegressor
from sklearn.linear_model._ridge import RidgeCV
from sklearn.linear_model._stochastic_gradient import SGDRegressor
from sklearn.svm._classes import SVR
from sklearn.linear_model._glm.glm import TweedieRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.model_selection import KFold

n_fold = 5
r2 = 0
predicted_times = np.zeros(len(X_test))
cv = KFold(n_splits=n_fold, shuffle=True, random_state=420)

estimators = [('est0', MLPRegressor(max_iter=10000)),
 ('est1', BayesianRidge()),
 ('est2', RidgeCV()),
 ('est3', KNeighborsRegressor()),
 ('est4', ElasticNetCV()),
 ('est5', NuSVR(max_iter=100000))]

festimator = GradientBoostingRegressor()

model = StackingRegressor(estimators=estimators,final_estimator=festimator)

# model.fit(train_pca, ertime)


for fold_n, (train_index, valid_index) in enumerate(cv.split(X)):
    print('\nFold', fold_n, 'started at', ctime())

    X_train = X.iloc[train_index,:]
    X_valid = X.iloc[valid_index,:]
    
    Y_train = Y.iloc[train_index]
    Y_valid = Y.iloc[valid_index]
    
    model.fit(X_train,Y_train)
    r2 += model.score(X_valid,Y_valid)
    predicted_times += model.predict(X_test)
        
r2 /= n_fold
predicted_times /= n_fold

print('R2 {:3f} min {:}'.format(r2,predicted_times.min()))


Fold 0 started at Wed Dec  2 18:04:25 2020

Fold 1 started at Wed Dec  2 18:04:57 2020

Fold 2 started at Wed Dec  2 18:05:40 2020

Fold 3 started at Wed Dec  2 18:06:23 2020

Fold 4 started at Wed Dec  2 18:07:06 2020
R2 0.936720 min -1.6513822439804584


In [7]:
predicted_times = timescaler.inverse_transform(predicted_times)
predicted_times2 = predicted_times
#MLP R2 0.938642 min -1.7697310024988808

In [8]:
submission = pd.DataFrame({
        "segment_id": test_id,
        "time_to_eruption": predicted_times2
    })

submission.to_csv('submission_4.csv', index=False)